In [4]:
import os
os.chdir("../")


In [5]:
%pwd

'c:\\Users\\rayjohndp\\Desktop\\Projects\\DS_Python_Portfolio\\Telecom_Churn_Project'

In [6]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/rjdp07/Telecom_Churn_Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "rjdp07"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0a9816f8c59f4bd161a99945ec3bb97d3faa61cf"

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    Xtest_data_path: Path
    ytest_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from churnPrediction.constants import *
from churnPrediction.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGB
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            Xtest_data_path = config.Xtest_data_path,
            ytest_data_path = config.ytest_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/rjdp07/Telecom_Churn_Project.mlflow"
        )

        return model_evaluation_config



In [13]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from churnPrediction import logger

In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy =accuracy_score(actual,pred)
        return accuracy
    
    def log_into_mlfow(self):

        X_test = pd.read_csv(self.config.Xtest_data_path)
        y_test = pd.read_csv(self.config.ytest_data_path)

        model = joblib.load(self.config.model_path)
        
        #mlflow.set_registry_uri(self.config.mlflow_uri)
        
        mlflow.end_run()

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            model_accuracy = self.eval_metrics(y_test,predicted_qualities)

            scores = {"accuracy":model_accuracy}
            save_json(path = Path(self.config.metric_file_name), data = scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("accuracy",model_accuracy)
            tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme
            logger.info("run here???")
            #Model registry does not work with file store
            if tracking_url_type_store != "file":
                #Register the model
                #There are other ways to use model registry, which depends on the use case,
                #please refer to the doc for more information
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBoostModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlfow()
except Exception as e:
    raise e

[2024-06-20 14:13:25,400: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-20 14:13:25,401: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-20 14:13:25,404: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-20 14:13:25,405: INFO: common: created directory at: artifacts]
[2024-06-20 14:13:25,405: INFO: common: created directory at: artifacts/model_evaluation]
[2024-06-20 14:13:25,753: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
[2024-06-20 14:13:26,679: INFO: 334472148: run here???]


Registered model 'XGBoostModel' already exists. Creating a new version of this model...
2024/06/20 14:13:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBoostModel, version 10
Created version '10' of model 'XGBoostModel'.
